In [1]:
import pandas as pd
import numpy as np
from time import perf_counter
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


In [10]:
x_df, y_df = prep.shuffle_data(joined_df, rand, seed=seed)

In [11]:
def objective(h_params, X, y):
    print(h_params)
    model = SVR(
        kernel=h_params[0],
        C=h_params[1],
        epsilon=h_params[2],
        gamma=h_params[3],
        degree=h_params[4],
        coef0=h_params[5]
    )
    # Standardize all variables (except one-hots)
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    ttr = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    return -np.mean(cross_val_score(
        ttr,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))


In [12]:
space = [
    Categorical(["linear", "rbf", "poly"], name="kernel"),
    Real(1e-1, 1e1, name="C"),
    Real(1e-3, 1e-1, name="epsilon"),
    Real(1e-3, 1e-1, name="gamma"),
    Integer(1, 3, name="degree"),
    Real(-1, 1, name="coef0")
]
n_initial = 10 * len(space[0].categories)
n_calls = 50 * len(space[0].categories)

In [14]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
#     callback=save_callback
    callback=lambda x: opt.save_callback(x, "opt_svr_h_params_TEST.tsv", sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
['rbf', 4.860832066690728, 0.07997511248069224, 0.028987590498937163, 2, -0.9804853006583336]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.8209
Function value obtained: 565.4600
Current minimum: 565.4600
Iteration No: 2 started. Evaluating function at random point.
['poly', 6.923792863971573, 0.08706755969427699, 0.0275125783572481, 2, -0.4634373845765508]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.5617
Function value obtained: 648.8427
Current minimum: 565.4600
Iteration No: 3 started. Evaluating function at random point.
['rbf', 2.778095179642898, 0.05347613910877633, 0.018361893417211655, 2, 0.7822181916783213]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.5225
Function value obtained: 558.7666
Current minimum: 558.7666
Iteration No: 4 started. Evaluating function at random point.
['linear', 4.9937304017493425, 0.022017512235608248, 0.052566810001

KeyboardInterrupt: 